In [10]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import json
import csv
from itertools import zip_longest
import pandas as pd
import numpy as np
import datetime

In [28]:
def dubizzle_scraping():  
    
    start = datetime.datetime.now()
    print("Scraping Start ",start)
    
    rent = "property-for-rent"
    sale = "property-for-sale"
    commercial = "commercial"
    house = "residential"

    types_Dict = {1: [sale, house] ,2: [rent, house],3: [sale, commercial], 4: [rent, commercial]}

    categorys_residential = ['apartment', 'villahouse', 'townhouse', 'penthouse', 'residential-building', 'residential-floor',
                             'villa-compound']


    categorys_commercial = ['office', 'retail', 'industrial', 'staff-accomm', 'shop', 'warehouse', 'commercial-floor',
                            'commercial-building', 'commercial-villa', 'factory', 'showroom']
    

    categorys_commercial = ['showroom']

    dict_total = {}
    list_dict = []
    basic_url = "https://dubai.dubizzle.com/"

    counter = 0

    links = []
    all_links = []




    for type_num in types_Dict:

        page_num = 0
        nn = 0
        
        while (True):
            
            if type_num == 1 or type_num == 2:
                for x in categorys_residential:
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
                    url = (f"https://dubai.dubizzle.com/{types_Dict[type_num][0]}/{types_Dict[type_num][1]}/{x}/?page={page_num}")
                    result = requests.get(url, headers=headers)
                    src = result.content
                    soup = BeautifulSoup(src, "lxml")
                    links.append([url,type_num])
                    
            if type_num == 3 or type_num == 4:

                for x in categorys_commercial:
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
                    url = (f"https://dubai.dubizzle.com/{types_Dict[type_num][0]}/{types_Dict[type_num][1]}/{x}/?page={page_num}")
                    result = requests.get(url, headers=headers)
                    src = result.content
                    soup = BeautifulSoup(src, "lxml")
                    links.append([url,type_num])

            next_page_link = soup.find("a", {"class": "sc-e8rug5-3 iGAVu next_button"})
            
            if (next_page_link == None ):
                complet = datetime.datetime.now()            
                print("complet ", complet - start)
                break
            else:
                page_num += 1
                nn +=1
                print(f"Done---> {nn}")


    thousand = 1

    for link,cat_num in (links):

        nn += 1
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
        result = requests.get(link, headers=headers)
        soup = BeautifulSoup(result.content, 'lxml') 
        info = soup.find_all("div", {"class":"sc-15l4r6f-0"})

        for soup in info:

            add_link = soup.find("a").attrs['href']

    #         print(add_link)
            #--- Urls
            dict_total['url'] = add_link
            #----------------------------------------------------------------------------------------------------------

            #--- Unit
            try:
                u = soup.find("div", {"class":"_4b74b8bb"})
                dict_total['unit'] = x
            except:
                dict_total['unit'] = None

            #----------------------------------------------------------------------------------------------------------

            # Price

            if cat_num == 2 or cat_num == 4 :
                try:
                    p = soup.find("div", {"class":"sc-11jo8dj-0 iFVtrV"})
                    rent_price = int(p.find("div", {"data-testid": "listing-price"}).text.replace(",", "")) / 12
                    dict_total['price'] = round(rent_price)

                except:
                    dict_total['price'] = None
            else:
                try:
                    p = soup.find("div", {"class":"sc-11jo8dj-0 iFVtrV"})
                    dict_total['price'] = int(p.find("div", {"data-testid": "listing-price"}).text.replace(",", ""))
                except:
                    dict_total['price'] = None

          #----------------------------------------------------------------------------------------------------------

          # City

            try:
                c = soup.find("div", {"class":"sc-1barvlq-3 Tstvk"})
                dict_total['city'] = c.find("span", {"class":"MuiTypography-root MuiTypography-body1 MuiTypography-noWrap css-1g74ci9"}).text.split()[-1]

            except:
                dict_total['city'] = None

         #----------------------------------------------------------------------------------------------------------

         # Location1

            try:
                c = soup.find("div", {"class":"sc-1barvlq-3 Tstvk"})
                dict_total['Location_1'] = c.find("span", {"class":"MuiTypography-root MuiTypography-body1 MuiTypography-noWrap css-1g74ci9"}).text.split(',')[-2]

            except:
                dict_total['Location_1'] = None

         #----------------------------------------------------------------------------------------------------------

        # Location2

            try:
                c = soup.find("div", {"class":"sc-1barvlq-3 Tstvk"})
                dict_total['Location_2'] = c.find("span", {"class":"MuiTypography-root MuiTypography-body1 MuiTypography-noWrap css-1g74ci9"}).text.split(',')[-3]

            except:
                dict_total['Location_2'] = None

        #----------------------------------------------------------------------------------------------------------
        # addres

            try:
                c = soup.find("div", {"class":"sc-1barvlq-3 Tstvk"})
                dict_total['addres'] = c.find("span", {"class":"MuiTypography-root MuiTypography-body1 MuiTypography-noWrap css-1g74ci9"}).text
            except:
                dict_total['addres'] = None

        #----------------------------------------------------------------------------------------------------------

        #--- bedrooms

            beds = soup.find("div", {"class":"sc-1barvlq-0"})
            try:
                dict_total['bedrooms'] = beds.find("div", {"data-testid":"listing-bedrooms"}).text
            except:
                dict_total['bedrooms'] = 0

        #----------------------------------------------------------------------------------------------------------

        #--- bathrooms

            bat = soup.find("div", {"class":"sc-1barvlq-0"})
            try:
                dict_total['bathrooms'] = bat.find("div", {"data-testid":"listing-bathrooms"}).text
            except:
                dict_total['bathrooms'] = 0
        #---------------------------------------------------------------------------------------------------------- 

        #--- Size

            try:
                s = soup.find("div", {"class":"sc-1barvlq-0"})
                a_str = s.find("div", {"data-testid":"listing-size"}).text.replace(' sqft','')
                a_round =  (int(a_str.replace(",", ""))) * 0.0929
                dict_total['size'] = round(a_round)
            except:
                dict_total['size'] = None
        #----------------------------------------------------------------------------------------------------------

        # More INfo

            dict_total['web_name'] = "Dubizzle"
            dict_total['source'] = "scraped"
            try:
                dict_total['rent_or_sale'] = (types_Dict[cat_num][0])
            except:
                dict_total['rent_or_sale'] = None
            try:
                dict_total['type'] = (types_Dict[cat_num][1])
            except:
                dict_total['type'] = None


            list_dict.append(dict_total.copy())

            counter += 1
            if (counter == thousand*1000) :
                coun = datetime.datetime.now() 
                print(counter , coun - start)
                thousand+=1

        print(counter)

        df = pd.DataFrame.from_dict(list_dict)
        df.to_csv('showroom_sale.csv', index=False, header=True, encoding='utf-8-sig')
        
        #--------- Save Data To Csv File
        
        print("Scraped")
        end = datetime.datetime.now()
        print("Scraping End ",end)
        print("duration ",end - start)

    
dubizzle_scraping()

Scraping Start  2023-05-21 02:58:51.012191
complet  0:00:01.661740
3
Scraped
Scraping End  2023-05-21 02:58:53.065028
duration  0:00:02.052837


{'url': 'https://dubai.dubizzle.com/property-for-rent/commercial/office/2023/4/4/well-furnished-offices-newly-established-c-2-780/', 'unit': 'office', 'city': 'Dubai', 'Location_1': ' Business Bay', 'Location_2': 'Metropolis Tower', 'addres': 'Metropolis Tower, Business Bay, Dubai', 'bedrooms': 0, 'bathrooms': 0, 'size': 19, 'web_name': 'Bayut', 'source': 'scraped', 'rent_or_sale': 'property-for-rent', 'type': 'commercial', 'price': 3917}
